In [44]:
import os, glob, json, re
import pandas as pd

In [45]:
records = []

# Append data from all text files in the data_properties directory
for filepath in glob.glob(os.path.join("", "*.txt")):
    with open(filepath, 'r', encoding='utf-8-sig') as f:
        for line in f:
            records.append(json.loads(line))

df = pd.DataFrame(records)
df.describe(include='all')

,project_name,location,location_map
count,643,643,643
unique,642,15,154
top,Acacia Escalades,Quezon City\n ...,"BGC, Taguig, Metro Manila"
freq,2,112,39


In [46]:
# read csv file from another directory
df_listings = pd.read_csv("D:/HEDONIC-MODEL/data_listings/data-listings.csv", dtype=str)

# subset the DataFrame to include only relevant columns
df_listings = df_listings[['project_name', 'location']].dropna().drop_duplicates()

df_listings

,project_name,location
0,The Calinea Tower,"Grace Park East, Caloocan"
21,Torre De Florencia,"Grace Park West, Caloocan"
34,Hacienda Balai,"Kaligayahan, Quezon City"
35,Elevé Homes Camarin,"Camarin, Caloocan"
39,Torre De Florencia,"Monumento, Caloocan"
...,...,...
15577,Viceroy,"Pinagsama, Taguig"
15580,Cypress Towers,"Ususan, Taguig"
15582,Alta Spatial,"Karuhatan, Valenzuela"
15585,Isabelle de Valenzuela,"Marulas, Valenzuela"


In [47]:
# merge df and df_listings using project_name, add new rows for all missing from df
df_merged = df.merge(df_listings, on='project_name', how='outer')
df_merged


,project_name,location_x,location_map,location_y
0,100 West,NaN,NaN,"Pio Del Pilar, Makati"
1,100 West Makati,Makati\n ...,"Pio Del Pilar, Makati, Metro Manila",NaN
2,1001 Parkway Residences,NaN,NaN,"Muntinlupa, Metro Manila"
3,101 Newport BLVD,Manila\n ...,"Barangay 183, Pasay, Metro Manila",NaN
4,101 Xavierville,NaN,NaN,"Loyola Heights, Quezon City"
...,...,...,...,...
1287,dakotaresidences,Metro Manila,"Tugatog, Malabon, Metro Manila","Tugatog, Malabon"
1288,mckinley hill garden villas,Taguig\n ...,"Bagong Tanyag, Taguig, Metro Manila",NaN
1289,oriental gardens makati,Makati\n ...,"Bangkal, Makati, Metro Manila",NaN
1290,symfoni kamias,Quezon City\n ...,"Ramon Magsaysay, Quezon City, Metro Manila","East Kamias, Quezon City"


In [48]:
# apply string cleaning and corrections
df_merged['location_x'] = df_merged['location_x'].str.replace(r'\s{2,}', ' ', regex=True).str.replace(' , ', ', ').str.strip()

# check if the string in 'location_map' is not missing and contains the corresponding 'location_x' string, case-insensitive
df_merged['location_x_match'] = df_merged.apply(lambda row: str(row['location_x']).lower() in str(row['location_map']).lower(), axis=1)
df_merged['location_map_empty'] = df_merged['location_map'].notna()
df_merged['location_match'] = df_merged['location_x_match'] & df_merged['location_map_empty']
df_merged = df_merged.drop(columns=['location_x_match', 'location_map_empty'])

df_merged

,project_name,location_x,location_map,location_y,location_match
0,100 West,NaN,NaN,"Pio Del Pilar, Makati",False
1,100 West Makati,"Makati, Metro Manila","Pio Del Pilar, Makati, Metro Manila",NaN,True
2,1001 Parkway Residences,NaN,NaN,"Muntinlupa, Metro Manila",False
3,101 Newport BLVD,"Manila, Metro Manila","Barangay 183, Pasay, Metro Manila",NaN,False
4,101 Xavierville,NaN,NaN,"Loyola Heights, Quezon City",False
...,...,...,...,...,...
1287,dakotaresidences,Metro Manila,"Tugatog, Malabon, Metro Manila","Tugatog, Malabon",True
1288,mckinley hill garden villas,"Taguig, Metro Manila","Bagong Tanyag, Taguig, Metro Manila",NaN,True
1289,oriental gardens makati,"Makati, Metro Manila","Bangkal, Makati, Metro Manila",NaN,True
1290,symfoni kamias,"Quezon City, Metro Manila","Ramon Magsaysay, Quezon City, Metro Manila","East Kamias, Quezon City",True


In [49]:
# Save the DataFrame to a CSV file
df_merged.to_csv("data-properties-for-correction.csv", index=False, encoding='utf-8-sig')